# This notebook shows the interpolation process for each video

In [1]:
import os, sys, pandas as pd, numpy as np

# Import library with current code functions
sys.path.append(os.path.join("..", "lib"))
import general_functions as gf, files_paths as fp

## Defining some functions


In [2]:
def index_frames_generate(VD_INFO, VD_FEATURE):
    
    # Get vide information 
    VIDEO_ID = VD_FEATURE['video_id'][0]
    FRAME_SEQ = np.array(VD_FEATURE['frame_seq'], dtype=np.int16)
    TIME_STEP_FR = VD_INFO['time_step_fr'][0]
    ORIGIN_VID = VD_INFO['origin_vid'][0]
    
    # Get the maximum value of frame_seq
    frame_seq_max = FRAME_SEQ[-1]
    
    # Make a dataframe with a video_id column and n rows (n=maximum value of frame_seq)
    FRAMES_DTFRAME = pd.DataFrame(data={'video_id': VIDEO_ID}, index=pd.RangeIndex(0,frame_seq_max+1,1), dtype=np.int16)
    FRAMES_DTFRAME['frame_seq'] = range(0,frame_seq_max+1)
    
    # Fill the collumn fr_exists with 1 in the indexes that the frame_seq number exists
    FRAMES_DTFRAME.loc[FRAME_SEQ, 'fr_exists'] = 1
    
    # Replace NaN values with zeros in 
    FRAMES_DTFRAME['fr_exists'] = FRAMES_DTFRAME['fr_exists'].fillna(0).astype(int)
    
    # Filter Dataframe to find discontinuous intervals
    FRAMES_DTFRAME['frames_total'] = (FRAMES_DTFRAME.fr_exists.diff(1) != 0).cumsum()
    FRAMES_DTFRAME = pd.DataFrame({'fr_exists' : FRAMES_DTFRAME.groupby('frames_total').fr_exists.first(),
                  'frame_seq_init' : FRAMES_DTFRAME.groupby('frames_total').frame_seq.first(), 
                  'frame_seq_end': FRAMES_DTFRAME.groupby('frames_total').frame_seq.last(),
                  'frames_total' : FRAMES_DTFRAME.groupby('frames_total').size()}).reset_index(drop=True)
    
    # Add  origin_vid collumn
    FRAMES_DTFRAME.insert(0, 'origin_vid', ORIGIN_VID)
    FRAMES_DTFRAME.insert(0, 'video_id', VIDEO_ID)
    FRAMES_DTFRAME.insert(6, 'time_step_fr', TIME_STEP_FR)

    return FRAMES_DTFRAME

In [3]:
def color_interpolated_rows(row):
    if row['fr_exists'] == 0 and row['frames_total'] < 5:
        return ['color: red']*len(row)
    else: return [''] * len(row) 

## Reading CSV's

In [4]:
# D for local video or Y for online video
ORIGIN = "D"
VIDEO_ID = 21

VD_INFO = gf.READ_CSV_FILE(gf.get_video_path(ORIGIN, VIDEO_ID, fp.VD_INFO))
VD_FEATURES_L1 = gf.READ_CSV_FILE(gf.get_video_path(ORIGIN, VIDEO_ID, fp.VD_FEATURES_L2))
VD_FEATURES_L2 = gf.READ_CSV_FILE(gf.get_video_path(ORIGIN, VIDEO_ID, fp.VD_FEATURES_L2))

## Shows the video frames with it's discontinuities

In [5]:
index_frames_generate(VD_INFO, VD_FEATURES_L1).style.apply(color_interpolated_rows, axis=1)

,video_id,origin_vid,fr_exists,frame_seq_init,frame_seq_end,frames_total,time_step_fr
0,21,D,0,0,10,11,0.033367
1,21,D,1,11,5148,5138,0.033367


## Shows the video frames interpolated

In [6]:
index_frames_generate(VD_INFO, VD_FEATURES_L2)

,video_id,origin_vid,fr_exists,frame_seq_init,frame_seq_end,frames_total,time_step_fr
0,21,D,0,0,10,11,0.033367
1,21,D,1,11,5148,5138,0.033367
